In [1]:
import pandas as pd
import numpy as np
import redis

In [2]:
hostname = "redis-13392.c325.us-east-1-4.ec2.cloud.redislabs.com"
portnumber = 13392
password = "zzq1ZvmrXn80N8aeJwvaXl7PjNh6BTQf"

r = redis.StrictRedis(host=hostname,
                      port=portnumber,
                      password=password)

In [3]:
r.ping()

True

In [4]:
r.keys()

[b'books:2', b'academy:register', b'books:1', b'order', b'ze']

In [5]:
np_file = np.load("data_frame_students_teacher.npz", allow_pickle=True)
np_file

NpzFile 'data_frame_students_teacher.npz' with keys: arr_0, arr_1

In [6]:
x_values = np_file["arr_0"]
col_names = np_file["arr_1"]
df = pd.DataFrame(x_values, columns=col_names)
df

,name,role,embedding
0,Joe,Student,"[-0.73383904, -1.1926562, 0.16253585, -1.23318..."
1,Jon,Student,"[0.013050548, -0.28796571, 0.49172014, 0.22333..."
2,Martin,Teacher,"[0.4679354, -0.67061454, 0.07022456, -0.941410..."
3,Rob,Student,"[-1.1936102, 0.09637058, 0.1618901, -0.2628968..."
4,Tyrion,Teacher,"[-0.57296205, -0.48193663, -0.5918181, -0.2621..."


In [7]:
df['name-role'] = df['name'] + '@' + df['role']
df

,name,role,embedding,name-role
0,Joe,Student,"[-0.73383904, -1.1926562, 0.16253585, -1.23318...",Joe@Student
1,Jon,Student,"[0.013050548, -0.28796571, 0.49172014, 0.22333...",Jon@Student
2,Martin,Teacher,"[0.4679354, -0.67061454, 0.07022456, -0.941410...",Martin@Teacher
3,Rob,Student,"[-1.1936102, 0.09637058, 0.1618901, -0.2628968...",Rob@Student
4,Tyrion,Teacher,"[-0.57296205, -0.48193663, -0.5918181, -0.2621...",Tyrion@Teacher


In [8]:
records = df[["name-role", "embedding"]].to_dict(orient="records")
records

[{'name-role': 'Joe@Student',
  'embedding': array([-0.73383904, -1.1926562 ,  0.16253585, -1.2331842 , -1.2056078 ,
          1.4246587 ,  0.12676767,  0.8672677 ,  2.3327632 , -1.6866531 ,
         -1.4974121 ,  0.5575733 ,  0.02753367,  0.8235652 , -2.4880857 ,
         -0.20051603, -0.37315437, -0.4818151 ,  0.88589126,  2.0865817 ,
          0.6169242 ,  0.15318187,  0.4121096 ,  0.23579016, -0.6030558 ,
          0.00700516,  0.24056539, -0.10220034,  1.4318129 , -0.53018564,
          1.9643724 , -0.31678194, -0.14278388, -0.25507742,  0.03597362,
          0.5241716 ,  0.06184995,  0.51577455, -0.5696997 , -0.07525869,
          0.17679973, -0.3476495 ,  0.23252186, -1.0385602 , -0.144134  ,
          0.1050173 , -0.12202599, -0.04052607,  0.9093212 , -0.56257164,
          1.4910431 , -0.17721896,  0.7929796 , -0.70894444, -1.3021965 ,
         -0.09129665,  0.73372364, -1.4542825 , -1.358828  , -0.43697438,
         -0.15376857,  0.71929216, -0.71491605,  0.04550492,  0.12582

In [9]:
records[0]["name-role"]

'Joe@Student'

In [10]:
for record in records:
    name_role = record["name-role"]
    vector = record["embedding"]

    #convert numpy array to bytes
    vector_bytes = vector.tobytes() 

    r.hset(name="academy:register", key=name_role, value=vector_bytes)

In [11]:
r.keys()

[b'books:2', b'academy:register', b'books:1', b'order', b'ze']

In [12]:
import cv2
from insightface.app import FaceAnalysis

In [13]:
face_app = FaceAnalysis(name="buffalo_sc",
                        root="insightface_model",
                        providers=["CPUExecutionProvider"])

face_app.prepare(ctx_id=0, det_size=(640,640), det_thresh=0.5)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_sc\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_sc\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [15]:
person_name = input("Enter your name: ")

trials = 3
for trial in range(trials):
    
    role = input('''
    1. Student
    2. Teacher
                 
    Enter either 1 or 2
    ''')

    if role in ("1", "2"):
        if role == "1":
            role = "Student"
        elif role == 2:
            role = "Teacher"
        break
    else:
        print("Invalid Entry. Try again")
        if trial == 2:
            print("Exceeds maximum number of trials")

key = person_name + "@" + role

print("\n")
print("Your name: ", person_name)
print("Your role: ", role)
print("Key: ", key)

Enter your name:  Athif

    1. Student
    2. Teacher
                 
    Enter either 1 or 2
     1




Your name:  Athif
Your role:  Student
Key:  Athif@Student


In [29]:
cap = cv2.VideoCapture(0)
face_embeddings = []
sample = 0

while True:
    ret, frame = cap.read()
    if ret == False:
        print("Unable to read the camera")
        break

    results = face_app.get(frame, max_num=1)
    for res in results:
        sample += 1
        x1, y1, x2, y2 = res["bbox"].astype(int)
        cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 1)

        embeddings = res["embedding"]
        face_embeddings.append(embeddings)

    if sample >= 200:
        break
    
    cv2.imshow("frame", frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [30]:
len(face_embeddings)

200

In [38]:
x_mean = np.array(face_embeddings).mean(axis=0)

In [39]:
x_mean.shape

(512,)

In [41]:
x_mean_bytes = x_mean.tobytes()

In [43]:
r.hset(name="academy:register", key=key, value=x_mean_bytes)

1